#Introduction

Title: Exam MCQ Generator

#Installs

In [ ]:
!pip install -q python-dotenv

!pip install -q langchain_experimental
!pip install -q langchain

!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requ

In [ ]:
!pip install -q pydantic

Using the rapidocr-onnxruntime package we can extract images as text as well

In [ ]:
!pip install pypdf rapidocr-onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.8 MB/s eta 0:00:00


#Imports

In [ ]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

#Data Source

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def read_text_file(path):
  with open(path, 'r', encoding='utf-8') as f:
    text = f.read()

  return text

In [ ]:
texts_path = '/content/drive/MyDrive/datasets/exam_mcq_generator/texts'

text_path_01 = os.path.join(texts_path, '01.txt')
text_path_02 = os.path.join(texts_path, '02.txt')
text_path_03 = os.path.join(texts_path, '03.txt')
text_path_04 = os.path.join(texts_path, '04.txt')
text_path_05 = os.path.join(texts_path, '05.txt')

In [73]:
pdfs_path = '/content/drive/MyDrive/datasets/exam_mcq_generator/pdfs'

pdf_path_the_time_machine = os.path.join(pdfs_path, 'The-Time-Machine.pdf')
pdf_path_pdf_test_01 = os.path.join(pdfs_path, 'pdf_test_01.pdf')

#Env

In [ ]:
env_path = '/content/drive/MyDrive/credentials/data-analytics-demo/.env'

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(env_path)

True

In [ ]:
GOOGLE_PALM_API_KEY = os.environ['GOOGLE_PALM_API_KEY']
HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']
OPEN_AI_API_KEY = os.environ['OPEN_AI_API_KEY']

# GOOGLE_PALM_API_KEY, HUGGINGFACE_API_KEY, OPEN_AI_API_KEY

#Model

In [ ]:
from pydantic import BaseModel, Field

class MCQModel(BaseModel):
  question: str = Field(description="This is the question text")
  options: list[str] = Field(description="This is a list of multiple choices or options avalible")
  correct_option: str = Field(description="This is the correct choice or option")
  difficulty_level: str = Field(description="This is the difficulty level of the question from one of the three modes: easy, medium, and hard")

In [ ]:
class MCQListModel(BaseModel):
  mcq_list: list[MCQModel] = Field(description="A list of multiple choice questions")

In [ ]:
from langchain.output_parsers import PydanticOutputParser

pydantic_parser = PydanticOutputParser(pydantic_object=MCQListModel)

format_instructions = pydantic_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"mcq_list": {"title": "Mcq List", "description": "A list of multiple choice questions", "type": "array", "items": {"$ref": "#/definitions/MCQModel"}}}, "required": ["mcq_list"], "definitions": {"MCQModel": {"title": "MCQModel", "type": "object", "properties": {"question": {"title": "Question", "description": "This is the question text", "type": "string"}, "options": {"title": "Options", "description": "This is a list of multiple choices or options avalible", "type": "array", "items": {"type": "string"}}, "correct_option": {"tit

In [ ]:
langchain_llm = ChatOpenAI(openai_api_key=OPEN_AI_API_KEY, model_name="gpt-3.5-turbo-0613", verbose=False,)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [ ]:
prompt = (
    "Following are blocks of text picked at random from a document:"
    "\n{texts}"
    "\nGenerate {n} non-repeating multiple choice questions along with their respective correct options from this text."
    "\nBy default the difficulty level is set to {difficulty_level}, if it is mix then randomly select difficulty"
    "\nAlso, return the difficulty level of the question from one of the three modes: easy, medium, and hard."
    "\nDo not use any outside information."
    "\n{format_instructions}"
)

In [ ]:
prompt_template = ChatPromptTemplate.from_template(prompt)

prompt_template

ChatPromptTemplate(input_variables=['difficulty_level', 'format_instructions', 'n', 'texts'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['difficulty_level', 'format_instructions', 'n', 'texts'], template='Following are blocks of text picked at random from a document:\n{texts}\nGenerate {n} non-repeating multiple choice questions along with their respective correct options from this text.\nBy default the difficulty level is set to {difficulty_level}, if it is mix then randomly select difficulty\nAlso, return the difficulty level of the question from one of the three modes: easy, medium, and hard.\nDo not use any outside information.\n{format_instructions}'))])

In [ ]:
llm_chain = LLMChain(llm=langchain_llm, prompt=prompt_template, output_key="result")

In [ ]:
def get_docs_text(path, chunk_size=500, chunk_overlap=0):
  """
  note:
    * When chunk_overlap=0, it will try not cut sentences, which is good for out case
  """

  text = read_text_file(path)

  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap,
      length_function=len,
      is_separator_regex=False,
  )

  docs = text_splitter.create_documents([text])

  return docs

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

def get_docs_pdf(path, chunk_size=500, chunk_overlap=0):
  """
  note:
    * When chunk_overlap=0, it will try not cut sentences, which is good for out case
  """

  loader = PyPDFLoader(path, extract_images=True)

  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap,
      length_function=len,
      is_separator_regex=False,
  )

  # by default it was doing page split
  docs = loader.load_and_split(text_splitter=text_splitter)

  return docs

In [ ]:
from langchain.callbacks import get_openai_callback as get_langchain_openai_callback

import random

class InvalidFileExtensionError(Exception):
    pass

def get_exam(file_path, n=3, difficulty_level="mix"):
  """
  validation: total_docs <= n
  """

  if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")

  _, file_extension = os.path.splitext(file_path)

  if file_extension == '.txt':
    docs = get_docs_text(file_path)
  elif file_extension == '.pdf':
    docs = get_docs_pdf(file_path)
  else:
    raise InvalidFileExtensionError(f"Invalid file extension: {file_extension}")

  if n > len(docs):
    raise ValueError(f"Too many questions specified, must be less then {len(docs)+1}")

  rand_docs = random.sample(docs, n)

  texts = "".join([f"Text Block {i+1}:\n{doc.page_content}\n" for i, doc in enumerate(rand_docs)])

  with get_langchain_openai_callback() as cb_langchain:
    response = llm_chain.invoke({"texts": texts, "n": n, "difficulty_level": difficulty_level, "format_instructions": format_instructions})

  return {"mcqs_parsed": MCQListModel.parse_raw(response['result']).mcq_list, 'cb_langchain': cb_langchain}

# get_exam(text_01)

#Testing Text

In [ ]:
def display_exam(exam):

  for i in exam['mcqs_parsed']:

    print(f"question: {i.question}\n")

    for index, j in enumerate(i.options):
      print(f"{index+1}. {j}")

    print(f"\ncorrect: {i.correct_option}\n")

    print(f"difficulty: {i.difficulty_level}\n\n")

  print(exam['cb_langchain'])

In [ ]:
# exam_01 = get_exam(text_01, n=10)

In [ ]:
exam_01 = get_exam(text_path_01, n=5, difficulty_level='hard')

In [ ]:
display_exam(exam_01)

question: What did Seraphina discover about the archipelago?

1. It was home to mythical creatures
2. It held the key to a realm between reality and dreams
3. It was a lost civilization with forgotten magic
4. It had boundaries of time and space intertwined

correct: It held the key to a realm between reality and dreams

difficulty: hard


question: What did Eldridge Haven stand as a testament to?

1. The enduring power of curiosity
2. The enduring power of courage
3. The enduring power of uncharted territories
4. The enduring power of curiosity, courage, and uncharted territories

correct: The enduring power of curiosity, courage, and uncharted territories

difficulty: medium


question: What did Seraphina take with her on her journey?

1. A weathered satchel, a compass, and the shimmering artifact
2. A compass and the ancient map
3. A weathered satchel and the shimmering artifact
4. A compass, the ancient map, and the shimmering artifact

correct: A weathered satchel, a compass, and 

In [ ]:
exam_02 = get_exam(text_path_02, n=5)

In [ ]:
display_exam(exam_02)

question: What emerged as SynthEra became more ubiquitous?

1. A) A vibrant subculture
2. B) A new era of human expression
3. C) The intersection of technology and creativity
4. D) The boundaries between the tangible and the virtual

correct: A) A vibrant subculture

difficulty: easy


question: What were some of the ethical considerations with SynthEra?

1. A) Privacy and security
2. B) Potential misuse of intimate access to the human mind
3. C) Innovation and responsibility
4. D) All of the above

correct: D) All of the above

difficulty: medium


question: What were some of the possibilities unlocked by SynthEra?

1. A) Interacting with augmented reality overlays
2. B) Accessing vast databases and analyzing complex data sets
3. C) Communicating with others in a purely mental dialogue
4. D) All of the above

correct: D) All of the above

difficulty: medium


question: Where did SynthEra originate from?

1. A) Quantum Dynamics Innovations
2. B) Laboratories of Quantum Dynamics Innovat

In [ ]:
exam_03 = get_exam(text_path_03, n=5)

In [ ]:
display_exam(exam_03)

question: What is the Collatz conjecture?

1. A mathematical puzzle that has been solved
2. A philosophical question about the nature of truth
3. A conundrum in abstract mathematics
4. A cycle of numbers that repeats endlessly

correct: A conundrum in abstract mathematics

difficulty: medium


question: What is the core rule of the Collatz conjecture?

1. Divide any positive integer by 2
2. Triple any positive integer and add 1
3. Take the square root of any positive integer
4. Add 1 to any positive integer

correct: Divide any positive integer by 2

difficulty: easy


question: What is the significance of the Collatz conjecture?

1. It provides a solution to a mathematical puzzle
2. It deepens our understanding of the beauty of numbers
3. It challenges the core of mathematical understanding
4. It reveals the secrets of the infinite realms of possibility

correct: It challenges the core of mathematical understanding

difficulty: medium


question: What happens to the sequence in the Co

In [ ]:
exam_04 = get_exam(text_path_04, n=5)

In [ ]:
display_exam(exam_04)

question: What does poetry do in the human experience?

1. Forge connections between hearts and intellects
2. Offer refuge for introspection and contemplation
3. Distill intricate emotions into a paltry ensemble of words
4. Transcend linguistic and temporal boundaries
5. Manifest in sundry configurations and idioms

correct: D

difficulty: easy


question: What is the primary function of poetry?

1. To evoke, inspire, and transcend
2. To serve as an emollient for the psyche
3. To distill intricate emotions into a paltry ensemble of words
4. To connect cultures and epochs
5. To manifest in sundry configurations and idioms

correct: A

difficulty: easy


question: What is the art of compaction in poetry?

1. The ability to distill intricate emotions into a paltry ensemble of words
2. The transcendence of linguistic demarcations
3. The ability to connect cultures and epochs
4. The manifestation in sundry configurations and idioms
5. The weaving of incantations that transport readers

corr

In [ ]:
exam_05 = get_exam(text_path_05, n=5)

In [ ]:
display_exam(exam_05)

question: What is the focus of ongoing research on LLMs?

1. Refining statistical approaches
2. Addressing ethical concerns
3. Exploring novel applications
4. All of the above

correct: All of the above

difficulty: easy


question: Which experiment in the mid-20th century marked a pioneering effort in computer-based language translation?

1. The Georgetown-IBM experiment
2. The statistical language model experiment
3. The advent of neural networks
4. None of the above

correct: The Georgetown-IBM experiment

difficulty: easy


question: Which approach in the late 20th century led to a paradigm shift in natural language processing?

1. Rule-based systems
2. Statistical approaches
3. Computational linguistics
4. Neural networks

correct: Statistical approaches

difficulty: medium


question: What was the key breakthrough in the evolution of LLMs in the 21st century?

1. Introduction of recurrent neural networks
2. Rise of computational linguistics
3. Development of rule-based systems
4.

#Testing PDF

In [ ]:
# get_docs_pdf(pdf_path_the_time_machine)

In [ ]:
exam_06 = get_exam(pdf_path_the_time_machine, n=5, difficulty_level='hard')

/usr/local/lib/python3.10/dist-packages/langchain_community/document_loaders/parsers/pdf.py:124: UserWarning: Unknown PDF Filter!
  warnings.warn("Unknown PDF Filter!")


In [ ]:
display_exam(exam_06)

question: What did the narrator see after entering the tunnel?

1. A large open space
2. A narrow tunnel
3. A vast arched cavern
4. Utter darkness

correct: A vast arched cavern

difficulty: medium


question: What was the narrator's interpretation of the things they saw?

1. The whole earth had become a garden
2. There were signs of agriculture
3. The whole earth had become a desert
4. The whole earth had become a forest

correct: The whole earth had become a garden

difficulty: medium


question: What did the narrator do to escape from the Morlocks?

1. Lit another match and retreated to the tunnel
2. Ran towards the open space
3. Fought against the Morlocks
4. Hid in the vast arched cavern

correct: Lit another match and retreated to the tunnel

difficulty: medium


question: What did the narrator observe about the wells?

1. They were connected to tall towers
2. They had no bottom
3. They emitted a flicker in the air
4. They were filled with water

correct: They emitted a flicker i

#Experiments

In [88]:
from langchain_community.document_loaders import PyPDFLoader

def get_docs_pdf_exp(path, chunk_size=500, chunk_overlap=0):
  """
  note:
    * When chunk_overlap=0, it will try not cut sentences, which is good for out case
  """

  loader = PyPDFLoader(path, extract_images=True)

  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap,
      length_function=len,
      is_separator_regex=False,
  )

  # by default it was doing page split
  docs = loader.load_and_split(text_splitter=text_splitter)

  return docs

In [89]:
docs_01 = get_docs_pdf_exp(pdf_path_pdf_test_01)

In [90]:
for index, doc in enumerate(docs_01):
  print(f'Doc: {index+1}')
  print(doc.page_content)
  print(doc.metadata)
  print('\n')

Doc: 1
PDF-T est-01
1P D F - T e s t - 0 1
It
the
best of
:was
times, it
the
was
 worst
of times, it
the a
was
age
of wisdom, it
the
was
age of foolishness...
{'source': '/content/drive/MyDrive/datasets/exam_mcq_generator/pdfs/pdf_test_01.pdf', 'page': 0}


Doc: 2
PDF-T est-01
2
"LIVELIFEWITHNO EXCLISES
TRAVELWITHNOREGRET"
-OSCAR WILDESintony
120
ABIEIALUAX
AlignHorizontally
Left
三Center
三Right
三Justify
Align Vertically
不Top
Middle
Bottom
{'source': '/content/drive/MyDrive/datasets/exam_mcq_generator/pdfs/pdf_test_01.pdf', 'page': 1}


Doc: 3
PDF-T est-01
3
{'source': '/content/drive/MyDrive/datasets/exam_mcq_generator/pdfs/pdf_test_01.pdf', 'page': 2}


Doc: 4
PDF-T est-01
4
{'source': '/content/drive/MyDrive/datasets/exam_mcq_generator/pdfs/pdf_test_01.pdf', 'page': 3}


